In [2]:
import boto3
import uuid
from boto3.dynamodb.conditions import Key

# For a Boto3 client.
ddb = boto3.resource('dynamodb',
                          aws_access_key_id="anything",
                          aws_secret_access_key="anything", endpoint_url='http://localhost:8000')

table = ddb.Table('table')


In [7]:
# table = ddb.create_table(
#     TableName='table',  # replace with your table name
#     KeySchema=[
#         {
#             'AttributeName': 'PK',  # primary key attribute name
#             'KeyType': 'HASH'  # primary key type
#         },
#         {
#             'AttributeName': 'SK',  # sort key attribute name
#             'KeyType': 'RANGE'  # sort key type
#         }
#     ],
#     AttributeDefinitions=[
#         {
#             'AttributeName': 'PK',  # primary key attribute name
#             'AttributeType': 'S'  # 'S' for string
#         },
#         {
#             'AttributeName': 'SK',  # sort key attribute name
#             'AttributeType': 'S'  # 'S' for string
#         },
#     ],
#     ProvisionedThroughput={
#         'ReadCapacityUnits': 5,  # replace with your read capacity units
#         'WriteCapacityUnits': 5  # replace with your write capacity units
#     }
# )

In [26]:

#create_company
def create_company():
    company_id = f"company_{str(uuid.uuid4())}"
    company = {
                "id": company_id,
                "name": "New Company"
            }
    res = table.put_item(
        Item={
            'PK': f"COMPANY",
            'SK': f"COMPANY#{company_id}",
            'company': company
        }
    )
    return company
create_company()

{'id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b', 'name': 'New Company'}

In [3]:
def get_all_companies():
    res = table.query(
        KeyConditionExpression=Key('PK').eq('COMPANY') & Key('SK').begins_with('COMPANY#')
    )
    return [i['company'] for i in res.get('Items', [])]
get_all_companies()

[{'name': 'New Company', 'id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b'},
 {'name': 'lemon corp', 'id': 'company_5129f337-105a-41b1-ba8b-0bc6c4fcd508'},
 {'name': 'lemon corp', 'id': 'company_b7356a30-9c49-40d1-a60a-aafc34e6313c'}]

In [40]:
company_id = "company_2598dce3-08f4-4c92-93b4-17c36844c91b"
user_data = {
    "name":"asdfasdf",
    "email": "333@gmail.com"
}

def create_user(company_id, user_data):
    user_id = f"user_{str(uuid.uuid4())}"
    user_data['id'] = user_id
    user_data['company_id'] = company_id
    res = table.put_item(
        Item={
            'PK': f"COMPANY#{company_id}",
            'SK': f"USER#{user_id}",
            'user': user_data
        }
    )
    return user_data
create_user(company_id, user_data) 

{'name': 'asdfasdf',
 'email': '333@gmail.com',
 'id': 'user_4e9a802f-4265-4bc7-a707-af09f3009117',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b'}

In [4]:
COMPANY_ID = "company_2598dce3-08f4-4c92-93b4-17c36844c91b"
USER_ID = "user_4e9a802f-4265-4bc7-a707-af09f3009117"
def get_user(company_id, user_id):
    response = table.get_item(
        Key={
            'PK': f"COMPANY#{company_id}",
            'SK': f"USER#{user_id}",
        }
    )
    user_data = response.get('Item', {}).get('user')
    return user_data
get_user(COMPANY_ID, USER_ID)

{'name': 'asdfasdf',
 'id': 'user_4e9a802f-4265-4bc7-a707-af09f3009117',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
 'email': '333@gmail.com'}

In [10]:
def delete_user(company_id, user_id):
    response = table.delete_item(
        Key={
            'PK': f"COMPANY#{company_id}",
            'SK': f"USER#{user_id}"
        }
    )
    return response
delete_user(COMPANY_ID, "user_60f94b9a-5982-4689-92ad-1c91f5e5ee65")

{'ResponseMetadata': {'RequestId': '0c944560-2303-4050-9f21-bfeb64b04b61',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jul 2023 21:22:45 GMT',
   'x-amzn-requestid': '0c944560-2303-4050-9f21-bfeb64b04b61',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(9.4.48.v20220622)'},
  'RetryAttempts': 0}}

In [7]:
def get_all_users(company_id):
    res = table.query(
        KeyConditionExpression=Key('PK').eq(f"COMPANY#{company_id}") & Key('SK').begins_with("USER#")
    )
    return [i['user'] for i in res.get('Items', [])]

get_all_users('company_2598dce3-08f4-4c92-93b4-17c36844c91b')

[{'name': 'tom',
  'email': '2@gmail.com',
  'id': 'user_4cd6cd3f-2962-4a9e-9cca-975a2d9fba2a'},
 {'name': 'asdfasdf',
  'id': 'user_4e9a802f-4265-4bc7-a707-af09f3009117',
  'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
  'email': '333@gmail.com'}]

In [33]:
query_data = {
    "MEO": True,
    "summary": "152423",
    "details": "213451234"
}

def create_query(company_id, user_id, query):
    query_id = f"query_{str(uuid.uuid4())}"
    if query['MEO']:
        sk = f"MEO#{user_id}#QUERY#{query_id}"
    else:
        sk = f"MEO#FALSE#QUERY#{query_id}"
    
    query['id'] = query_id
    query['company_id'] = company_id
    res = table.put_item(
        Item={
            'PK': f"COMPANY#{company_id}#QUERY",
            'SK': sk,
            'query': query
        }
    )
    return query
    

create_query(COMPANY_ID, "user_4cd6cd3f-2962-4a9e-9cca-975a2d9fba2a", query_data)

{'MEO': True,
 'summary': '152423',
 'details': '213451234',
 'id': 'query_45748250-dab2-4552-ba7f-f8658fc477e1',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b'}

In [9]:
def get_all_queries(company_id, user_id):
    res = table.query(
        KeyConditionExpression=Key('PK').eq(f"COMPANY#{company_id}#QUERY") & Key('SK').begins_with(f"MEO#{user_id}#QUERY#")
    )
    queries = res.get("Items", [])
    res = table.query(
        KeyConditionExpression=Key('PK').eq(f"COMPANY#{company_id}#QUERY") & Key('SK').begins_with(f"MEO#FALSE#QUERY#")
    )
    queries.append(res.get("Items", []))
    return [i['query'] for i in res.get('Items', [])]
get_all_queries(COMPANY_ID, USER_ID)

[{'summary': '152423',
  'details': '213451234',
  'id': 'query_6d66a543-4205-438c-b0d9-08f08fcf13dc',
  'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
  'MEO': False},
 {'summary': 'tqwerfdf',
  'details': 'asdads',
  'id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e',
  'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
  'MEO': False}]

In [21]:
QUERY_ID = "query_f888990e-1606-410f-aeb7-85262fef7d5e"
QUERY_MEO = "query_336f1577-53e6-44c5-9e08-57eafe4edcfa"
def get_query(company_id, user_id, query_id):
    res = table.get_item(
        Key={
            'PK': f"COMPANY#{company_id}#QUERY",
            'SK': f"MEO#FALSE#QUERY#{query_id}",
        }
    )
    if res.get("Item"):
        return res['Item']['query']
    res = table.get_item(
        Key={
            'PK': f"COMPANY#{company_id}#QUERY",
            'SK': f"MEO#{user_id}#QUERY#{query_id}",
        }
    )
    if res.get("Item"):
        return res['Item']['query']
    
get_query(COMPANY_ID, USER_ID, QUERY_ID)

{'summary': 'tqwerfdf',
 'details': 'UPDATED',
 'id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
 'MEO': False}

In [20]:
def update_query(company_id, user_id, query_id, query):
    if query['MEO']:
        sk = f"MEO#{user_id}#QUERY#{query_id}"
    else:
        sk = f"MEO#FALSE#QUERY#{query_id}"
    res = table.put_item(
        Item={
            'PK': f"COMPANY#{company_id}#QUERY",
            'SK': sk,
            'query': query
        }
    )
    return query

new_query = {'summary': 'tqwerfdf',
 'details': 'UPDATED',
 'id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
 'MEO': False}

update_query(COMPANY_ID, USER_ID, QUERY_ID, new_query)

{'summary': 'tqwerfdf',
 'details': 'UPDATED',
 'id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e',
 'company_id': 'company_2598dce3-08f4-4c92-93b4-17c36844c91b',
 'MEO': False}

In [26]:
activity_1 = {"type": "comment", "comment":"lemons", "from": "jeff"}
activity_2 = {"type": "comment", "comment":"asdfawf", "from": "31dff"}
import ulid

def create_query_activity(query_id, activity):
    activity_id = f"activity_{str(ulid.new())}"
    
    activity['id'] = activity_id
    activity['query_id'] = query_id
    
    res = table.put_item(
        Item={
            'PK': f"QUERY#{query_id}",
            'SK': f"ACTIVITY#{activity_id}",
            'activity': activity
        }
    )
    return activity

create_query_activity(QUERY_ID, activity_2)

{'type': 'comment',
 'comment': 'asdfawf',
 'from': '31dff',
 'id': 'activity_01H4TS4Q72GPX414SFXD37X38Z',
 'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'}

In [30]:
def get_activity(query_id):
    res = table.query(
        KeyConditionExpression=Key('PK').eq(f"QUERY#{query_id}") & Key('SK').begins_with(f"ACTIVITY#")
    )
    return [i['activity'] for i in res.get('Items', [])]
get_activity(QUERY_ID)
    
    

[{'comment': 'lemons',
  'from': 'jeff',
  'id': 'activity_01H4TQ69N7M2342MMGKNBCTSTN',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'lemons',
  'from': 'jeff',
  'id': 'activity_01H4TQ6R36QC3H28S1DAYWQ5Q7',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'asdfawf',
  'from': '31dff',
  'id': 'activity_01H4TQ7569HGXKRP6F8149D8SG',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'},
 {'comment': 'asdfawf',
  'from': '31dff',
  'id': 'activity_01H4TS4Q72GPX414SFXD37X38Z',
  'type': 'comment',
  'query_id': 'query_f888990e-1606-410f-aeb7-85262fef7d5e'}]